In [44]:
import pandas as pd

In [45]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)




In [46]:
with open('input.txt','r',encoding='utf-8') as f:
    lines=f.read().split('\n')

In [47]:
print(lines[155])

It's me!	Ich bin's.


In [48]:
len(lines)

169813

In [49]:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

In [50]:
num_samples = 10000
num_samples

10000

In [51]:
in_out=pd.read_csv('inout.csv')
input_texts=list(in_out['Input'])
target_texts=list(in_out['Output'])

for input_text,target_text in zip(input_texts,target_texts):
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [52]:
input_texts[155],target_texts[155]

('What is ecclesiology?', '\tthe doctrine of the church\n')

In [53]:
input_texts

['Where were begging children often targeted for being sold into slavery?',
 'What is the term for tranquility?',
 'Which country had successful public analog HDTV broadcasting?',
 'What does ftp stand for?',
 'Who defined mammalian as the crown group mammals?',
 'What did the PVA and KPA gain by using the tactics during the Third Phase Offensive',
 'What lake has some hotels around it?',
 'What was the title of the book written by Thomas Sowell?',
 'What is the estimated number of Buddhists in the world?',
 'What are the gametes produced by females called?',
 'How many people watched the first episode of American Idol?',
 'What is a football player shown before being ejected from a match?',
 'What was intended to be a platform for the development of business activities?',
 'What film led to Close Encounters?',
 'What does taking a culture of an infectious agent isolated from a patient allow?',
 'What has been a cause of mass parrot poisonings attributed to zinc?',
 'What amoun of the 

In [54]:
target_texts

['\tbus stations\n',
 '\tjhāna\n',
 '\tJapan\n',
 '\tFile Transfer Protocol\n',
 '\tTimothy Rowe\n',
 '\tSeoul\n',
 '\tLake Paranoá\n',
 '\tAffirmative Action Around the World: An Empirical Study\n',
 '\t488 million[web 1] and 535 million\n',
 '\teggs or ova\n',
 '\t9.9 million\n',
 '\ta red penalty card\n',
 '\tThe Towers Square\n',
 '\tFirelight\n',
 '\tdetailed identification\n',
 '\tjuices stored in galvanized cans\n',
 '\t0.97% to 2%\n',
 '\tThe Bolsheviks\n',
 '\ttwo\n',
 '\t1929\n',
 '\tU.S. Ambassador to Canada\n',
 '\tstanding wave\n',
 '\tS. pyogenes\n',
 '\tKorea and Vietnam\n',
 '\t1867\n',
 '\tEdward\n',
 '\tmonarchy\n',
 '\tLos Angeles\n',
 '\tthe Holy Roman Empire\n',
 "\tDesigners' Way\n",
 '\tover 30,000\n',
 '\tThe 24-pin double-sided connector\n',
 '\tall times\n',
 '\tfull-time undergraduate\n',
 '\thydrogen\n',
 '\tblood plasma fractions\n',
 '\tHayy ibn Yaqdhan (Philosophus Autodidactus)\n',
 '\t1992\n',
 '\tfour-year terms,\n',
 '\tthe Hellenistic age\n',
 '\tRon

In [55]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 10000
Number of unique input tokens: 130
Number of unique output tokens: 155
Max sequence length for inputs: 100
Max sequence length for outputs: 183


In [56]:
input_token_index = dict(
  [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
  [(char, i) for i, char in enumerate(target_characters)])

In [57]:
import numpy as np

encoder_input_data = np.zeros(
  (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
  dtype='float32')
decoder_input_data = np.zeros(
  (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
  dtype='float32')
decoder_target_data = np.zeros(
  (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
  dtype='float32')

In [58]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
    # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
          # decoder_target_data will be ahead by one timestep
          # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

In [59]:
import keras,tensorflow
from keras.models import Model
from keras.layers import Input,LSTM,Dense


In [60]:
batch_size = 64  # batch size for training
epochs = 100  # number of epochs to train for
latent_dim = 256  # latent dimensionality of the encoding space

In [61]:
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

In [62]:
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [63]:
model = Model(inputs=[encoder_inputs, decoder_inputs], 
              outputs=decoder_outputs)

In [64]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.summary()

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, None, 130)    0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            (None, None, 155)    0                                            
__________________________________________________________________________________________________
lstm_3 (LSTM)                   [(None, 256), (None, 396288      input_5[0][0]                    
__________________________________________________________________________________________________
lstm_4 (LSTM)                   [(None, None, 256),  421888      input_6[0][0]                    
                                                                 lstm_3[0][1]               

In [ ]:
 model.fit([encoder_input_data, decoder_input_data],
           decoder_target_data, batch_size=batch_size,
           epochs=epochs, validation_split=0.2)

Train on 8000 samples, validate on 2000 samples
Epoch 1/100
8000/8000 [==============================] - 26s 3ms/step - loss: 0.3780 - val_loss: 0.3672
Epoch 2/100
8000/8000 [==============================] - 31s 4ms/step - loss: 0.3408 - val_loss: 0.3379
Epoch 3/100
8000/8000 [==============================] - 30s 4ms/step - loss: 0.3121 - val_loss: 0.3122
Epoch 4/100
8000/8000 [==============================] - 27s 3ms/step - loss: 0.2987 - val_loss: 0.3038
Epoch 5/100
8000/8000 [==============================] - 25s 3ms/step - loss: 0.2899 - val_loss: 0.2947
Epoch 6/100
8000/8000 [==============================] - 25s 3ms/step - loss: 0.2829 - val_loss: 0.2889
Epoch 7/100
8000/8000 [==============================] - 25s 3ms/step - loss: 0.2770 - val_loss: 0.2828
Epoch 8/100
8000/8000 [==============================] - 26s 3ms/step - loss: 0.2719 - val_loss: 0.2789
Epoch 9/100
8000/8000 [==============================] - 25s 3ms/step - loss: 0.2674 - val_loss: 0.2749
Epoch 10/100
800

In [ ]:
# model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
# model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
# model.load_weights()


encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, state_h, state_c = decoder_lstm(
  decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model(
  [decoder_inputs] + decoder_states_inputs,
  [decoder_outputs] + decoder_states)


# reverse-lookup token index to turn sequences back to characters
reverse_input_char_index = dict(
  (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
  (i, char) for char, i in target_token_index.items())

def decode_sequence(input_seq):
    # encode the input sequence to get the internal state vectors.
    states_value = encoder_model.predict(input_seq)

    # generate empty target sequence of length 1 with only the start character
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index['\t']] = 1.

    # output sequence loop
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
          [target_seq] + states_value)

        # sample a token and add the corresponding character to the 
        # decoded sequence
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # check for the exit condition: either hitting max length
        # or predicting the 'stop' character
        if (sampled_char == '\n' or 
            len(decoded_sentence) > max_decoder_seq_length):
                stop_condition = True

        # update the target sequence (length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # update states
        states_value = [h, c]

    return decoded_sentence

In [ ]:
for seq_index in range(10):
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

In [ ]:
input_sentence = "How are you?"
test_sentence_tokenized = np.zeros(
  (1, max_encoder_seq_length, num_encoder_tokens), dtype='float32')
for t, char in enumerate(input_sentence):
    test_sentence_tokenized[0, t, input_token_index[char]] = 1.
print(input_sentence)
print(decode_sequence(test_sentence_tokenized))

In [ ]:
val_input_texts = []
val_target_texts = []
line_ix = 12000
for line in lines[line_ix:line_ix+10]:
    input_text, target_text = line.split('\t')
    val_input_texts.append(input_text)
    val_target_texts.append(target_text)

val_encoder_input_data = np.zeros(
  (len(val_input_texts), max([len(txt) for txt in val_input_texts]),
   num_encoder_tokens), dtype='float32')

for i, input_text in enumerate(val_input_texts):
    for t, char in enumerate(input_text):
        val_encoder_input_data[i, t, input_token_index[char]] = 1.

In [ ]:
for seq_index in range(10):
    input_seq = val_encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', val_input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence[:-1])
    print('Ground Truth sentence:', val_target_texts[seq_index])